In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import torch
#!pip install transformers
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
from scipy.special import softmax


     ---------------------------------------- 8.5/8.5 MB 14.4 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 15.6 MB/s eta 0:00:00
     -------------------------------------- 269.7/269.7 kB 8.4 MB/s eta 0:00:00
     ------------------------------------- 346.4/346.4 kB 10.8 MB/s eta 0:00:00
     ------------------------------------- 170.9/170.9 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Uninstalling fsspec-2022.7.1:
      Successfully uninstalled fsspec-2022.7.1


In [6]:
df = pd.read_csv('Labeled.csv', skipinitialspace=True)
pd.set_option('display.max_columns', None)

In [7]:
df.head()

,AG6019_neg,AG6019_pos,neg,neu,pos,compound,TB_Pol_label,TB_Sub_label,TB_Polarity,TB_Subjectivity,score,self_text,subreddit,ups,post_score,post_title,post_upvote_ratio,date,year,month,numerical_label,AG6019_polarity,VADER_polarity
0,0.926527,0.073473,0.254,0.746,0.000,-0.4696,neu,obj,0.000000,0.000000,1,Now Trump is mocking Haley's husband because.....,neoliberal,1,0,Discussion Thread,0.41,2024-02-10 20:51:38,2024,2,0,neg,neg
1,0.099621,0.900379,0.154,0.714,0.132,-0.7170,pos,sub,0.030556,0.407870,1,"nice, but my comment is that he has not been r...",conspiracy,1,304,You can’t have it both ways,0.71,2024-02-10 20:51:33,2024,2,1,pos,neg
2,0.559403,0.440597,0.000,1.000,0.000,0.0000,neu,obj,0.000000,0.000000,1,Could November be a bellwether for Tuesday’s e...,VoteDEM,1,27,"Daily Discussion Thread: February 10, 2024",0.93,2024-02-10 20:51:24,2024,2,2,neg,neu
3,0.061729,0.938271,0.000,1.000,0.000,0.0000,pos,obj,0.200000,0.200000,1,We really don't,neoliberal,1,0,Discussion Thread,0.41,2024-02-10 20:51:15,2024,2,0,pos,neu
4,0.532713,0.467287,0.219,0.678,0.103,-0.8806,neg,sub,-0.024554,0.439732,1,"And what makes you certain this is racism, and...",changemyview,1,0,CMV: the post text has a better definition of ...,0.18,2024-02-10 20:51:13,2024,2,3,neg,neg


Model source:  
https://huggingface.co/j-hartmann/emotion-english-distilroberta-base?text=This+movie+always+makes+me+cry..&library=true

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base")

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

c:\Users\Flores\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Flores\.cache\huggingface\hub\models--j-hartmann--emotion-english-distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [9]:
def emotion_scores_hartmann(x):
    encoded_text = tokenizer(x, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'Anger' : scores[0],
        'Disgust' : scores[1],
        'Fear' : scores[2],
        'Joy' : scores[3],
        'Neutral' : scores[4],
        'Sadness' : scores[5],
        'Surprise' : scores[6],
    }
    return scores_dict

In [10]:
df['self_text'] = df['self_text'].astype(str)
df['self_text'][1]

'nice, but my comment is that he has not been ruled incompetent to stand trial. You bring up Trump as if he might also be incompetent to stand trial for the very same reasons, lying to investigators to avoid self incrimination or legal entanglements.  \r\nBut the law doesn\'t distinguish any of this, its just that a prosecutor has discretion on who he thinks it is worth prosecuting, without evidence of intent or malice, he thinks cooperation will spare a likely "well meanning" but "elderly" man "with memory problems" from suffering a trial and aquital due to jury sympathy.  \r\nThats not what you are talking about, but Biden has lawyers and is following their advice no doubt.'

hartmann_results = {}

text = df['self_text'][1]
index_key = 1
hartmann_results[index_key] = emotion_scores_hartmann(text)

hartmann_results

In [18]:
hartmann_results = {}

for o, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['self_text']
        index_key = row.name
        hartmann_results[index_key] = emotion_scores_hartmann(text)
    except RuntimeError:
        print(f'Broke for row {index_key}')

  0%|          | 0/925060 [00:00<?, ?it/s]

KeyboardInterrupt: 

Output:
[[{'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'disgust', 'score': 0.0016119900392368436},
  {'label': 'fear', 'score': 0.0004138521908316761},
  {'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'neutral', 'score': 0.005764586851000786},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'surprise', 'score': 0.008528684265911579}]]


In [ ]:
hartmann = pd.DataFrame(hartmann_results).T
hartmann.index = df.index
df = pd.concat([hartmann, df], axis=1)
df.head()

In [ ]:
file_name = 'distilBERTed_Feelings.csv'

# Saving DataFrame to CSV file
#df.to_csv(file_name, index=False)